# Data World Xcrap 🌍

**Data Scraping IMDB Movie Site using Python**

# Step-by-Step python code

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Import Modules:**

In [ ]:
import pandas as pd
import numpy as np
import re
import lxml

from bs4 import BeautifulSoup
from requests import get
%matplotlib inline

**Get the link of the page for the top 100 most popular movies:**

In [ ]:
url = "https://www.imdb.com/search/title?count=100&title_type=feature,tv_series&ref_=nv_wl_img_2"

**Get the html of the page using requests.get(url) framework**

In [ ]:
page = get(url)

**Using Beautifulsoup and lxml to parse the html data**

In [ ]:
soup = BeautifulSoup(page.content, 'lxml')

**Get the element or tag that holds the movie contents**

In [ ]:
content = soup.find(id="main")

Get Article Title

soup.find("h1", class_="header")

finds the first line that has h1 tag and has a class header.

.text gets the text of that line or that element.

.replace("\n","") just erases \n.

In [ ]:
articleTitle = soup.find("h1", class_="header").text.replace("\n","")

In [ ]:
movieFrame = content.find_all("div", class_="lister-item mode-advanced")

In [ ]:
movieFirstLine = movieFrame[0].find("h3", class_="lister-item-header")

In [ ]:
movieTitle = movieFirstLine.find("a").text

In [ ]:
movieDate = re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text)

In [ ]:
movieRunTime = movieFrame[0].find("span", class_="runtime").text[:-4]

In [ ]:
movieGenre = movieFrame[0].find("span", class_="genre").text.rstrip().replace("\n","").split(",")

In [ ]:
movieRating = movieFrame[0].find("strong").text

In [ ]:
movieScore = movieFrame[0].find("span", class_="metascore unfavorable").text.rstrip()

In [ ]:
movieScore = movieFrame[0].find("span", class_="metascore favorable").text.rstrip()

In [ ]:
movieDesc = movieFrame[0].find_all("p", class_="text-muted")[-1].text.lstrip()

In [ ]:
movieCast = movieFrame[0].find("p", class_="")

In [ ]:
#Movie Director and Movie Stars
try:
    casts = movieCast.text.replace("\n","").split('|')
    casts = [x.strip() for x in casts]
    casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    movieDirector = casts[0]
    movieStars = [x.strip() for x in casts[1].split(",")]
except:
    casts = movieCast.text.replace("\n","").strip()
    movieDirector = np.nan
    movieStars = [x.strip() for x in casts.split(",")]

In [ ]:
movieNumbers = movieFrame[0].find_all("span", attrs={"name": "nv"})

In [ ]:
if len(movieNumbers) == 2:
    movieVotes = movieNumbers[0].text
    movieGross = movieNumbers[1].text
else:
    movieVotes = movieNumbers[0].text
    movieGross = np.nan

# IMDb class jupyter notebook code (full)

**Import Modules:**

In [ ]:
import lxml
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from requests import get

In [ ]:
url1 = "https://www.imdb.com/search/title?count=100&title_type=feature,tv_series&ref_=nv_wl_img_2"

In [ ]:
class IMDB(object):
	"""docstring for IMDB"""
	def __init__(self, url):
		super(IMDB, self).__init__()
		page = get(url)

		self.soup = BeautifulSoup(page.content, 'lxml')

	def articleTitle(self):
		return self.soup.find("h1", class_="header").text.replace("\n","")

	def bodyContent(self):
		content = self.soup.find(id="main")
		return content.find_all("div", class_="lister-item mode-advanced")

	def movieData(self):
		movieFrame = self.bodyContent()
		movieTitle = []
		movieDate = []
		movieRunTime = []
		movieGenre = []
		movieRating = []
		movieScore = []
		movieDescription = []
		movieDirector = []
		movieStars = []
		movieVotes = []
		movieGross = []
		for movie in movieFrame:
			movieFirstLine = movie.find("h3", class_="lister-item-header")
			movieTitle.append(movieFirstLine.find("a").text)
			movieDate.append(re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text))
			try:
				movieRunTime.append(movie.find("span", class_="runtime").text[:-4])
			except:
				movieRunTime.append(np.nan)
			movieGenre.append(movie.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
			try:
				movieRating.append(movie.find("strong").text)
			except:
				movieRating.append(np.nan)
			try:
				movieScore.append(movie.find("span", class_="metascore unfavorable").text.rstrip())
			except:
				movieScore.append(np.nan)
			movieDescription.append(movie.find_all("p", class_="text-muted")[-1].text.lstrip())
			movieCast = movie.find("p", class_="")

			try:
				casts = movieCast.text.replace("\n","").split('|')
				casts = [x.strip() for x in casts]
				casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
				movieDirector.append(casts[0])
				movieStars.append([x.strip() for x in casts[1].split(",")])
			except:
				casts = movieCast.text.replace("\n","").strip()
				movieDirector.append(np.nan)
				movieStars.append([x.strip() for x in casts.split(",")])

			movieNumbers = movie.find_all("span", attrs={"name": "nv"})

			if len(movieNumbers) == 2:
				movieVotes.append(movieNumbers[0].text)
				movieGross.append(movieNumbers[1].text)
			elif len(movieNumbers) == 1:
				movieVotes.append(movieNumbers[0].text)
				movieGross.append(np.nan)
			else:
				movieVotes.append(np.nan)
				movieGross.append(np.nan)

		movieData = [movieTitle, movieDate, movieRunTime, movieGenre, movieRating, movieScore, movieDescription,
							movieDirector, movieStars, movieVotes, movieGross]
		return movieData

In [ ]:
if __name__ == '__main__':
	site1 = IMDB(url1)
	print("Subject: ", site1.articleTitle())
	data = site1.movieData()
	for i in range(len(data)):
		print(data[i][:]) #Print the data

Subject:  Feature Film/TV Series(Sorted by Popularity Ascending) 
['365 Days', 'Da 5 Bloods', 'Artemis Fowl', 'The King of Staten Island', 'Dark', 'Knives Out', '13 Reasons Why', 'Game of Thrones', 'Community', 'Space Force', 'You Should Have Left', 'Marcella', 'Snowpiercer', 'The Office', 'The 100', 'Vikings', 'Supernatural', 'F Is for Family', 'The Order', "Grey's Anatomy", 'Palm Springs', 'Breaking Bad', 'Ozark', 'Hannibal', 'Money Heist', 'Avatar: The Last Airbender', 'Love, Victor', 'The Walking Dead', 'Tenet', 'Rick and Morty', 'Peaky Blinders', 'Yellowstone', 'Friends', 'The Last Kingdom', 'Modern Family', 'What We Do in the Shadows', 'Criminal Minds', 'Billions', 'The Sinner', 'The Salisbury Poisonings', 'Agents of S.H.I.E.L.D.', 'Stranger Things', 'The Politician', 'Uncut Gems', 'Outlander', 'Brooklyn Nine-Nine', "Schitt's Creek", 'Westworld', 'The Sopranos', 'Black Mirror', 'Lucifer', 'How to Get Away with Murder', 'The Big Bang Theory', 'Parasite', 'The Last Days of American